## Recommendation System

In [1]:
import firebase_admin
from firebase_admin import db,credentials
import main as recommender
import pandas as pd
from random import sample
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred, {'databaseURL': 'https://fir-demo-29d5b-default-rtdb.firebaseio.com'})

### Get data from Firebase Realtime DB

In [2]:
ref = db.reference('/')
data = ref.child('songs').order_by_child('id').get()
songs = []
for key in data:
    songs.append(data[key])

In [15]:
songs_df = pd.DataFrame(songs).dropna().drop_duplicates(['id'])

song_stat_df = songs_df.copy()
song_stat_df.drop(columns=['320kbps','album_id' ,'duration','has_lyrics' , 'image','perma_url', 'url', 'release_date', 'genre' ,'album','language' , 'subtitle','year'],inplace=True) #drop all of unnecessary columns 
# to convert string into list of strings 
def spliting(text):
  text=str(text).split()
  return text
song_stat_df['title'] = song_stat_df['title'].apply(spliting)
song_stat_df['artist'] = song_stat_df['artist'].apply(spliting)
song_stat_df['all_tags'] = song_stat_df['title'] + song_stat_df['artist'] 
song_stat_df.drop(columns=['title','artist'],inplace=True)
song_stat_df

,id,all_tags
0,-4eFAzI-7o0,"[THƯƠNG, THÌ, VỀ, ĐÂY, |, TIẾN, TỚI, |, Offici..."
1,-5q5mZbe3V8,"[BTS, (방탄소년단), 'Life, Goes, On', Official, MV,..."
2,-7sISWuTdj0,"[PHÚC, DU, -, yêu, anh, đi, mẹ, anh, bán, bánh..."
3,-E2u6exhzPE,"[MAKING, MY, WAY, Sơn, Tùng, M-TP]"
4,-GQg25oP0S4,"[SEVENTEEN, (세븐틴), '손오공', Official, MV, HYBE, ..."
...,...,...
1472,zlTIextYnyQ,"[KAI, 카이, 'Rover', MV, SMTOWN]"
1473,znlFu_lemsU,"[Mary, J., Blige, -, Family, Affair, (Official..."
1474,znvky0Uq8qc,"[Until, I, Found, You, Stephen, Sanchez]"
1475,zoC9Lxt2ZJU,"[Đừng, Chúc, Em, Hạnh, Phúc, (Lyrics, Video), ..."


In [16]:
def convert_lower(text):
  l=[]
  for item in text:
    l.append(item.lower())
  return l
song_stat_df['all_tags']=song_stat_df['all_tags'].apply(convert_lower)
song_stat_df

,id,all_tags
0,-4eFAzI-7o0,"[thương, thì, về, đây, |, tiến, tới, |, offici..."
1,-5q5mZbe3V8,"[bts, (방탄소년단), 'life, goes, on', official, mv,..."
2,-7sISWuTdj0,"[phúc, du, -, yêu, anh, đi, mẹ, anh, bán, bánh..."
3,-E2u6exhzPE,"[making, my, way, sơn, tùng, m-tp]"
4,-GQg25oP0S4,"[seventeen, (세븐틴), '손오공', official, mv, hybe, ..."
...,...,...
1472,zlTIextYnyQ,"[kai, 카이, 'rover', mv, smtown]"
1473,znlFu_lemsU,"[mary, j., blige, -, family, affair, (official..."
1474,znvky0Uq8qc,"[until, i, found, you, stephen, sanchez]"
1475,zoC9Lxt2ZJU,"[đừng, chúc, em, hạnh, phúc, (lyrics, video), ..."


In [17]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def steming(text):
  l=[]
  for i in text:
    l.append(ps.stem(i))
  return l

song_stat_df['all_tags'] = song_stat_df['all_tags'].apply(steming)
song_stat_df['all_tags'] = song_stat_df['all_tags'].apply(lambda x: " ".join(x))
song_stat_df

,id,all_tags
0,-4eFAzI-7o0,thương thì về đâi | tiến tới | offici music vi...
1,-5q5mZbe3V8,bt (방탄소년단) 'life goe on' offici mv hybe label
2,-7sISWuTdj0,phúc du - yêu anh đi mẹ anh bán bánh mì (mv of...
3,-E2u6exhzPE,make my way sơn tùng m-tp
4,-GQg25oP0S4,seventeen (세븐틴) '손오공' offici mv hybe label
...,...,...
1472,zlTIextYnyQ,kai 카이 'rover' mv smtown
1473,znlFu_lemsU,mari j. blige - famili affair (offici music vi...
1474,znvky0Uq8qc,until i found you stephen sanchez
1475,zoC9Lxt2ZJU,đừng chúc em hạnh phúc (lyric video) - thanh h...


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
all_tags = song_stat_df['all_tags']
data = vectorizer.fit_transform(all_tags)
print("Unique vocabulary: ", len(vectorizer.vocabulary_))
data.toarray()

Unique vocabulary:  3440


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(data)
similarity

array([[1.        , 0.01554383, 0.        , ..., 0.        , 0.00729383,
        0.        ],
       [0.01554383, 1.        , 0.02461316, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.02461316, 1.        , ..., 0.        , 0.08291845,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.00729383, 0.        , 0.08291845, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [19]:
def recommend(song):
    index = song_stat_df[song_stat_df['id'] == song].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    list_id = []
    for i in distances:
        if len(list_id) > 20:
            break
        id = song_stat_df.iloc[i[0]].id
        if id not in list_id:
            list_id.append(id)
    columns = ['320kbps','album','album_id','artist','duration','genre','has_lyrics','id','image','language','perma_url','release_date','subtitle','title','url','year']
    list_result = pd.DataFrame(columns= columns)
    for id in list_id:
        result = songs_df.query('id == @id')
        list_result = pd.concat([list_result, result], ignore_index=True).drop_duplicates(['id'])
    return list_result
    
recommend("jO2viLEW-1A")

,320kbps,album,album_id,artist,duration,genre,has_lyrics,id,image,language,perma_url,release_date,subtitle,title,url,year
0,false,Twenty One Pilots,UCnX0L9QiftAcWdzeBx31xCw,Twenty One Pilots,196,YouTube,false,oLeROuCMwj8,https://img.youtube.com/vi/oLeROuCMwj8/maxresd...,YouTube,https://www.youtube.com/watch?v=oLeROuCMwj8,2017-02-09 00:00:00.000,Twenty One Pilots - Topic,Heathens,https://rr2---sn-8qj-nbo6d.googlevideo.com/vid...,2017
1,false,twenty one pilots,UCBQZwaNPFfJ1gZ1fLZpAEGw,twenty one pilots,225,YouTube,false,Pw-0pbY9JeU,https://img.youtube.com/vi/Pw-0pbY9JeU/maxresd...,YouTube,https://www.youtube.com/watch?v=Pw-0pbY9JeU,2015-05-13 00:00:00.000,twenty one pilots,twenty one pilots - Ride (Official Video),https://rr4---sn-42u-nbozl.googlevideo.com/vid...,2015
2,false,Fueled By Ramen,UClVrJwcIy7saPcGc1nct80A,Fueled By Ramen,218,YouTube,false,UprcpdwuwCg,https://img.youtube.com/vi/UprcpdwuwCg/maxresd...,YouTube,https://www.youtube.com/watch?v=UprcpdwuwCg,2016-06-21 00:00:00.000,Fueled By Ramen,twenty one pilots: Heathens (from Suicide Squa...,https://rr1---sn-8qj-nbo6d.googlevideo.com/vid...,2016
3,false,Fueled By Ramen,UClVrJwcIy7saPcGc1nct80A,Fueled By Ramen,225,YouTube,false,pXRviuL6vMY,https://img.youtube.com/vi/pXRviuL6vMY/maxresd...,YouTube,https://www.youtube.com/watch?v=pXRviuL6vMY,2015-04-27 00:00:00.000,Fueled By Ramen,twenty one pilots: Stressed Out [OFFICIAL VIDEO],https://rr2---sn-8qj-nbo6d.googlevideo.com/vid...,2015
4,false,Calvin Harris,UCZ0Aezmtk-S2l8A9Ln-2lKw,Calvin Harris,215,YouTube,false,XWCPbRsMb0Q,https://img.youtube.com/vi/XWCPbRsMb0Q/maxresd...,YouTube,https://www.youtube.com/watch?v=XWCPbRsMb0Q,2018-04-05 00:00:00.000,Calvin Harris - Topic,One Kiss,https://rr8---sn-8qj-nboey.googlevideo.com/vid...,2018
5,false,Number One Hits,UCaVGVfuM8qWgyORw14-91sA,Number One Hits,280,YouTube,false,0UMAWfyxs4c,https://img.youtube.com/vi/0UMAWfyxs4c/maxresd...,YouTube,https://www.youtube.com/watch?v=0UMAWfyxs4c,2017-09-14 00:00:00.000,Number One Hits - Topic,Despacito,https://rr2---sn-8qj-nbo6d.googlevideo.com/vid...,2017
6,false,Ariana Grande,UC0076UMUgEng8HORUw_MYHA,Ariana Grande,197,YouTube,false,KEGZVv6KXL0,https://img.youtube.com/vi/KEGZVv6KXL0/maxresd...,YouTube,https://www.youtube.com/watch?v=KEGZVv6KXL0,2018-07-23 00:00:00.000,Ariana Grande - Topic,One Last Time,https://rr2---sn-8qj-nbo6d.googlevideo.com/vid...,2018
7,false,One Direction,UCREhun2tPd7aQnJczZM_Mww,One Direction,200,YouTube,false,d57qSDCxVA4,https://img.youtube.com/vi/d57qSDCxVA4/maxresd...,YouTube,https://www.youtube.com/watch?v=d57qSDCxVA4,2017-02-18 00:00:00.000,One Direction - Topic,What Makes You Beautiful,https://rr7---sn-8qj-nboey.googlevideo.com/vid...,2017
8,false,Maroon5VEVO,UCN1hnUccO4FD5WfM7ithXaw,Maroon5VEVO,223,YouTube,false,fwK7ggA3-bU,https://img.youtube.com/vi/fwK7ggA3-bU/maxresd...,YouTube,https://www.youtube.com/watch?v=fwK7ggA3-bU,2012-06-26 00:00:00.000,Maroon5VEVO,Maroon 5 - One More Night (Official Music Video),https://rr7---sn-42u-nbozz.googlevideo.com/vid...,2012
9,false,Teddy Swims,UCAtlzufZmPhkiDApWM4bTRQ,Teddy Swims,196,YouTube,false,jEylXhpABMs,https://img.youtube.com/vi/jEylXhpABMs/maxresd...,YouTube,https://www.youtube.com/watch?v=jEylXhpABMs,2020-07-16 00:00:00.000,Teddy Swims - Topic,You're Still The One,https://rr1---sn-8qj-nbo6d.googlevideo.com/vid...,2020


In [3]:
rec_sys = recommender.recommender_system()
rec_sys.data_preprocessing(songs=songs, stats= stats)

In [4]:
re = rec_sys.get_popularity()
out = pd.DataFrame(re)
out.head()

,320kbps,album,album_id,artist,duration,genre,has_lyrics,songId,image,language,perma_url,release_date,subtitle,title,url,year
0,false,Có Chắc Yêu Là Đây,UC3muIvzjhubNpJ4Pn_0kCQw,Sơn Tùng M-TP,215,YouTube,false,sTgsNeblHu8,https://lh3.googleusercontent.com/r6FYNr4_qCsa...,YouTube,https://www.youtube.com/watch?v=sTgsNeblHu8,2020-08-02 00:00:00.000,Song • Sơn Tùng M-TP • Có Chắc Yêu Là Đây • 3:36,Có Chắc Yêu Là Đây,https://rr1---sn-8qj-nbo6d.googlevideo.com/vid...,2020
1,true,Shape of You,3320300,Ed Sheeran,233,English,false,icJam_5l,https://c.saavncdn.com/126/Shape-of-You-Englis...,English,https://www.jiosaavn.com/song/shape-of-you/GQs...,2017-01-06,Ed Sheeran - Shape of You,Shape of You,https://aac.saavncdn.com/126/da7cde34b008294e1...,2017
2,true,÷,10436096,Ed Sheeran,263,English,false,6o8JoQ8b,https://c.saavncdn.com/286/WMG_190295851286-En...,English,https://www.jiosaavn.com/song/perfect/RgdTexthD1E,2017-03-03,Ed Sheeran - ÷,Perfect,https://aac.saavncdn.com/286/71bb6cc3391ddf619...,2017
3,true,Flowers,41619705,Miley Cyrus,200,English,false,c-6-mB8N,https://c.saavncdn.com/047/Flowers-English-202...,English,https://www.jiosaavn.com/song/flowers/E0VdHBlyD30,2023-01-13,Miley Cyrus - Flowers,Flowers,https://aac.saavncdn.com/047/985bd26eb8496d9a7...,2023
4,true,STAY,28263397,"Justin Bieber, The Kid Laroi",140,English,false,kd8JSDbB,https://c.saavncdn.com/895/Stay-English-2021-2...,English,https://www.jiosaavn.com/song/stay/GwxTeyd0VXE,2021-07-09,"Justin Bieber, The Kid Laroi - STAY",STAY,https://aac.saavncdn.com/895/9a3f48fb98e9dd0b2...,2021


In [5]:
pd.DataFrame(rec_sys.get_user_similarity('0c2983d0-f16d-11ed-9c88-514c38a48ebe'))

No. of unique songs for the user: 296
no. of unique songs in the training set: 795
Non zero values in cooccurence_matrix :99820


,320kbps,album,album_id,artist,duration,genre,has_lyrics,songId,image,language,perma_url,release_date,subtitle,title,url,year
0,true,Dandelions (slowed + reverb),29160455,"Ruth B., sped up + slowed",268,English,false,POhlVMwJ,https://c.saavncdn.com/839/Dandelions-slowed-r...,English,https://www.jiosaavn.com/song/dandelions-slowe...,2021-08-27,"Ruth B., sped up + slowed - Dandelions (slowed...",Dandelions (slowed + reverb),https://aac.saavncdn.com/839/2d3d560559fc503ae...,2021
1,true,Flowers,41619705,Miley Cyrus,200,English,false,c-6-mB8N,https://c.saavncdn.com/047/Flowers-English-202...,English,https://www.jiosaavn.com/song/flowers/E0VdHBlyD30,2023-01-13,Miley Cyrus - Flowers,Flowers,https://aac.saavncdn.com/047/985bd26eb8496d9a7...,2023
2,false,土岐麻子,UC9Lb-o-O-H35yTqFLXrlCnQ,土岐麻子,297,YouTube,false,ti62cJ_3oEM,https://img.youtube.com/vi/ti62cJ_3oEM/maxresd...,YouTube,https://www.youtube.com/watch?v=ti62cJ_3oEM,2020-08-31 00:00:00.000,土岐麻子,土岐麻子 / 「HOME」Music Video,https://rr1---sn-hvcp4vox8o-nbos.googlevideo.c...,2020
3,false,Amplifier,UCoLH6RkTGYdqW0qAfd5yO8Q,Amplifier,493,YouTube,false,46SGSGti2xQ,https://img.youtube.com/vi/46SGSGti2xQ/maxresd...,YouTube,https://www.youtube.com/watch?v=46SGSGti2xQ,2018-06-11 00:00:00.000,Amplifier - Topic,Matmos,https://rr1---sn-8qj-nbo66.googlevideo.com/vid...,2018
4,false,HIEUTHUHAI,UCh9j7NC6jVhHREbmqd9C7YA,HIEUTHUHAI,228,YouTube,false,r96eaATJWW8,https://img.youtube.com/vi/r96eaATJWW8/maxresd...,YouTube,https://www.youtube.com/watch?v=r96eaATJWW8,2023-04-19 00:00:00.000,HIEUTHUHAI - Topic,KHÔNG THỂ SAY,https://rr1---sn-8qj-nbo6d.googlevideo.com/vid...,2023
5,false,buitruonglinh,UCd23_I7g0mes9U5nL1XaPfw,buitruonglinh,213,YouTube,false,T3GHC43K5ZI,https://img.youtube.com/vi/T3GHC43K5ZI/maxresd...,YouTube,https://www.youtube.com/watch?v=T3GHC43K5ZI,2022-07-08 00:00:00.000,buitruonglinh - Topic,Dù Cho Mai Về Sau (Acoustic Version),https://rr8---sn-8qj-nboey.googlevideo.com/vid...,2022
6,false,Mono Official,UCi_RaMQ4Hhdml1Cff4SCpQA,Mono Official,265,YouTube,false,CHw1b_1LVBA,https://img.youtube.com/vi/CHw1b_1LVBA/maxresd...,YouTube,https://www.youtube.com/watch?v=CHw1b_1LVBA,2022-08-18 00:00:00.000,Mono Official,MONO - Waiting For You (Album 22 - Track No.10),https://rr5---sn-8qj-nbo67.googlevideo.com/vid...,2022
7,false,RodWave,UCenjunBhBhvKjfDAESnoppw,RodWave,208,YouTube,false,6dW4d0IcQ3k,https://img.youtube.com/vi/6dW4d0IcQ3k/maxresd...,YouTube,https://www.youtube.com/watch?v=6dW4d0IcQ3k,2023-03-30 00:00:00.000,RodWave,Rod Wave - Fight The Feeling (Official Video),https://rr5---sn-42u-nbozs.googlevideo.com/vid...,2023
8,false,Adam Christopher,UCw8h_nufRgtbdZX7enuXBFQ,Adam Christopher,184,YouTube,false,5HKeq3x6YPI,https://img.youtube.com/vi/5HKeq3x6YPI/maxresd...,YouTube,https://www.youtube.com/watch?v=5HKeq3x6YPI,2019-03-24 00:00:00.000,Adam Christopher - Topic,Iris (Acoustic),https://rr5---sn-42u-nbozs.googlevideo.com/vid...,2019
9,false,DJSnakeVEVO,UCGkSXL5saM8y3VvJD_PdMFQ,DJSnakeVEVO,205,YouTube,false,euCqAq6BRa4,https://img.youtube.com/vi/euCqAq6BRa4/maxresd...,YouTube,https://www.youtube.com/watch?v=euCqAq6BRa4,2016-11-29 00:00:00.000,DJSnakeVEVO,DJ Snake - Let Me Love You ft. Justin Bieber,https://rr1---sn-8qj-nbo6d.googlevideo.com/vid...,2016


## API

In [ ]:
import service as sv
self_sv = sv.firebase_service()
self_sv.get_data_and_preprocessing()

In [ ]:
self_sv.random_update_stats()